# Reddit Spark Streaming Consumer
This notebook receives Reddit posts/comments from a socket, stores them to a Spark table, and computes metrics such as reference counts, TF-IDF top words, and sentiment analysis.

In [10]:
!pip install textblob

In [11]:
from textblob import TextBlob
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

### Set-up of Spark Streaam Consumer and Data Schema structure.
##### See command to initialize spark server inside code cell. 

In [12]:
spark = SparkSession.builder.appName('RedditConsumer').getOrCreate()

# Command to run spark server on docker to plug into kernel for running notebook
# docker run -it -p 4040:4040 -p 8080:8080 -p 8081:8081 -p 8888:8888 -p 5432:5432 --cpus=2 --memory=2048m -h spark -w /mnt/host_home/ pyspark_container jupyter-lab --ip 0.0.0.0 --port 8888 --no-browser --allow-root

HOST = 'host.docker.internal'
PORT = 9998

schema = StructType([
    StructField('type', StringType()),
    StructField('subreddit', StringType()),
    StructField('id', StringType()),
    StructField('text', StringType()),
    StructField('created_utc', DoubleType()),
    StructField('author', StringType())
])

raw_lines = (spark
    .readStream
    .format('socket')
    .option('host', HOST)
    .option('port', PORT)
    .load())

json_df = raw_lines.select(F.from_json(F.col('value'), schema).alias('data')).select('data.*')

25/06/08 10:37:31 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


### Write data to memory

In [13]:
query_memory = (json_df
    .writeStream
    .outputMode('append')
    .format('memory')
    .queryName('raw')
    .start())

25/06/08 10:37:34 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7b68e808-f0f3-4f46-8728-861dd54a4c13. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/06/08 10:37:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


IllegalArgumentException: Cannot start query with name raw as a query with that name is already active in this SparkSession

### Get reference to users, subreddits and URLs

In [8]:
# Get reference to users, subreddits, and URLs in the text by using regex
user_refs = F.regexp_extract_all("text", r'/u/[^\s]+')
subreddit_refs = F.regexp_extract_all("text", r'/r/[^\s]+')
url_refs = F.regexp_extract_all("text", r'https?://[^\s]+')

### Create dataframes of references on a sliding window basis.

In [9]:
# get the count of each type of reference and tag them with a created timestamp
# for time based filtering and aggregation
refs_df = json_df.select(
    F.col('created_utc').cast('timestamp').alias('created_ts'),
    F.size(user_refs).alias('user_ref_count'),
    F.size(subreddit_refs).alias('subreddit_ref_count'),
    F.size(url_refs).alias('url_ref_count')
)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `/u/[^\s]+` cannot be resolved. Did you mean one of the following? [`author`, `subreddit`, `id`, `text`, `type`].;
'Project [cast(created_utc#8 as timestamp) AS created_ts#81, size(regexp_extract_all(text#7, '/u/[^\s]+, 1), true) AS user_ref_count#82, size(regexp_extract_all(text#7, '/r/[^\s]+, 1), true) AS subreddit_ref_count#83, size(regexp_extract_all(text#7, 'https?://[^\s]+, 1), true) AS url_ref_count#84]
+- Project [data#2.type AS type#4, data#2.subreddit AS subreddit#5, data#2.id AS id#6, data#2.text AS text#7, data#2.created_utc AS created_utc#8, data#2.author AS author#9]
   +- Project [from_json(StructField(type,StringType,true), StructField(subreddit,StringType,true), StructField(id,StringType,true), StructField(text,StringType,true), StructField(created_utc,DoubleType,true), StructField(author,StringType,true), value#0, Some(Etc/UTC)) AS data#2]
      +- StreamingRelationV2 org.a

In [ ]:
# get the total references per time window (60 seconds with a 5 second slide)
windowed_refs = (refs_df
    .withWatermark('created_ts', '1 minute')
    .groupBy(F.window('created_ts', '60 seconds', '5 seconds'))
    .sum('user_ref_count', 'subreddit_ref_count', 'url_ref_count')
)

In [ ]:
# Write the windowed reference counts to an in-memory table and to Parquet files
ref_query = (windowed_refs
    .writeStream
    .outputMode('update')
    .format('console')
    .option('truncate', False)
    .start())

### Function to extract references to users, subreddits and URLs

In [ ]:
def extract_reference_counts(batch_df):
    refs = (batch_df.select(
                F.regexp_extract_all('text', r'/u/\w+').alias('users'),
                F.regexp_extract_all('text', r'/r/\w+').alias('subs'),
                F.regexp_extract_all('text', r'https?://[^\s]+').alias('urls'))
            .select(
                F.size('users').alias('user_refs'),
                F.size('subs').alias('sub_refs'),
                F.size('urls').alias('url_refs')))
    refs_summary = refs.groupBy().sum('user_refs', 'sub_refs', 'url_refs')
    return refs_summary

### Function to compute TF-IDF and find top 10 most important words in the window.

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF

def compute_tfidf(batch_df):
    tokenizer = Tokenizer(inputCol='text', outputCol='words')
    words = tokenizer.transform(batch_df)
    remover = StopWordsRemover(inputCol='words', outputCol='filtered')
    filtered = remover.transform(words)
    hashingTF = HashingTF(inputCol='filtered', outputCol='rawFeatures', numFeatures=10000)
    featurized = hashingTF.transform(filtered)
    idf = IDF(inputCol='rawFeatures', outputCol='features')
    idf_model = idf.fit(featurized)
    tfidf = idf_model.transform(featurized)

    zipped = tfidf.select(F.explode(F.arrays_zip('filtered','features')).alias('z'))
    word_scores = zipped.select(F.col('z.filtered').alias('word'), F.col('z.features').alias('score'))
    top_words = word_scores.groupBy('word').agg(F.max('score').alias('score')).orderBy(F.desc('score')).limit(10)
    
    return top_words


### TextBlob function to achieve sentiment analysis of text.

In [ ]:
@udf(returnType=DoubleType())
def sentiment_udf(text):
    return TextBlob(text).sentiment.polarity if text else 0.0

#### Batch Processing of Streaming Data.
- TODO:
    - Requires references in window created previously
    - Requires top 10 words in TF-IDF
    - Write data to processed memory

In [ ]:
def process_batch(batch_df):
    batch_df.cache()
    batch_df.createOrReplaceTempView('current_batch')
    count = batch_df.count()
    print(f'Processing batch with {count} records')

    refs_summary = extract_reference_counts(batch_df)
    refs_summary.show(truncate=False)
    refs_summary.createOrReplaceTempView('batch_references')

    top_words = compute_tfidf(batch_df)
    top_words.show(truncate=False)
    top_words.createOrReplaceTempView('batch_tfidf')

    sentiment_scores = batch_df.withColumn('sentiment', sentiment_udf(F.col('text')))
    sentiment_scores.createOrReplaceTempView('batch_sentiment')


In [ ]:
process_query = (json_df
    .writeStream
    .foreachBatch(process_batch)
    .start())

### Kill Spark Instance

In [ ]:
spark.stop()

### Terminating all streams.

In [ ]:
# Await termination of all streams
for q in [query_memory, ref_query, process_query]:
    q.awaitTermination()

NameError: name 'ref_query' is not defined